In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
sc = pyspark.SparkContext(appName="Assignment")

## Part 1: Data Analysis ##

In [3]:
trips = sc.textFile('hdfs://master:9000/yellow_tripdata_1m.csv')
vendors = sc.textFile('hdfs://master:9000/yellow_tripvendors_1m.csv')

### a) Average trip duration per hour it started ###

In [4]:
from datetime import datetime

def strip_time(time):
    return datetime.strptime(time, '%Y-%m-%d %H:%M:%S')

In [5]:
avg_trp_dur = trips.map(lambda line: line.split(",")[1:3])\
        .map(lambda lst: (strip_time(lst[0]), strip_time(lst[1])))\
        .map(lambda lst: (lst[0].hour, lst[1]-lst[0]))\
        .mapValues(lambda x: (x, 1))\
        .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
        .map(lambda lst: (lst[0], lst[1][0].total_seconds()/(60*lst[1][1])))

In [6]:
print(avg_trp_dur.top(24))

[(23, 13.95847112523272), (22, 14.231797625637356), (21, 13.510855327392882), (20, 13.57589963636424), (19, 14.221208805985727), (18, 15.29045374860119), (17, 16.510825654408613), (16, 17.213072069374675), (15, 30.223498632126276), (14, 16.523138380789476), (13, 15.553918733195129), (12, 15.130881322885683), (11, 14.935821221905567), (10, 14.657939169698276), (9, 14.67010641976562), (8, 14.627504543367822), (7, 13.395006418527384), (6, 12.487420237563239), (5, 13.275583221175415), (4, 13.799857931121963), (3, 13.322282520526887), (2, 13.0356355926767), (1, 13.975069898907133), (0, 14.01779373736224)]


In [7]:
avg_trp_dur = avg_trp_dur.sortByKey(ascending=True)\
            .map(lambda tpl: {"HourOfDay":tpl[0], "AverageTripTime":tpl[1]})
fieldnames=["HourOfDay", "AverageTripTime"]

In [8]:
import csv
from io import StringIO

def writeRecords(records):
    """Write out CSV lines"""
    output = StringIO()
    writer = csv.DictWriter(output, fieldnames=fieldnames, delimiter='\t')
    for record in records:
        writer.writerow(record)
    return [output.getvalue()]

In [9]:
!hadoop fs -rm -r /average_trip.csv 
avg_trp_dur.mapPartitions(writeRecords).saveAsTextFile("hdfs://master:9000/average_trip.csv")

rm: `/average_trip.csv': No such file or directory


### b) Max amount paid per vendor ####

In [10]:
vendors_key_val = vendors.map(lambda line: line.split(",")[0:2])\
                        .map(lambda lst: (lst[0],lst[1]))

In [11]:
trips_key_val = trips.map(lambda line: line.split(",")[:])\
                    .map(lambda lst: (lst[0],lst[7]))

In [12]:
joinRDD = vendors_key_val.join(trips_key_val)\
                    .map(lambda tpl: tpl[1])\
                    .reduceByKey(max)

In [13]:
print(joinRDD.top(2))

[('2', '99.99'), ('1', '995.3')]


In [14]:
max_by_vndr = joinRDD.sortByKey(ascending=True)\
            .map(lambda tpl: {"VendorID":tpl[0], "MaxAmountPaid":tpl[1]})
fieldnames=["VendorID", "MaxAmountPaid"]

In [15]:
!hadoop fs -rm -r /max_amount.csv 
max_by_vndr.mapPartitions(writeRecords).saveAsTextFile("hdfs://master:9000/max_amount.csv")

rm: `/max_amount.csv': No such file or directory


## Part 2: Machine Learning ##

In [16]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array

In [17]:
ml_data = trips.map(lambda line: line.split(",")[3:5])\
                .map(lambda lst: array([float(lst[0]), float(lst[1])]))

In [18]:
init_model = KMeansModel(ml_data.take(5))

In [19]:
clusters = KMeans.train(ml_data, 5, maxIterations=3, initializationMode="kmeans||", initialModel=init_model)

In [20]:
clusters.clusterCenters

[array([-78.50386634,  40.61111272]),
 array([-0.0001304 ,  0.00050874]),
 array([-73.95393671,  40.69903131]),
 array([-73.99216792,  40.74268109]),
 array([-73.96113226,  40.77182523])]

In [21]:
i = 1
centroids = []
for c in clusters.clusterCenters:
    centroids.append((i, c))
    i += 1

In [22]:
p_centroids = sc.parallelize(centroids)
p_centroids = p_centroids.map(lambda tpl: {"ID":tpl[0], "Centroid":tpl[1]})
fieldnames=["ID", "Centroid"]

In [23]:
!hadoop fs -rm -r /centroids.csv 
p_centroids.mapPartitions(writeRecords).saveAsTextFile("hdfs://master:9000/centroids.csv")

rm: `/centroids.csv': No such file or directory


## Part 3: Page Rank ##


In [24]:
pages = sc.textFile('hdfs://master:9000/web-Google.txt')

In [25]:
edges = pages.filter(lambda line: line[0]!='#')\
            .map(lambda line: line.split('\t'))\
            .map(lambda lst: (int(lst[0]), int(lst[1])))

In [26]:
N = 875713
p0 = 0.5
d = 0.85

constant = 1-d/N

iterations = 5

In [27]:
links = edges.distinct().groupByKey().mapValues(list)

In [28]:
ranks = links.mapValues(lambda v: 0.5)

In [29]:
for it in range(iterations):    
    contribs = links.join(ranks)\
                   .map(lambda tpl: tpl[1])\
                   .flatMap(lambda tpl: map(lambda url: (url, tpl[1]/len(tpl[0])), tpl[0]))
    
    ranks = contribs.reduceByKey(lambda x,y: x + y).mapValues(lambda v: constant + d * v)

In [30]:
ranks = ranks.sortByKey(ascending=True)\
            .map(lambda tpl: {"Page ID":tpl[0], "PageRank":tpl[1]})
fieldnames=["Page ID", "PageRank"]

In [31]:
!hadoop fs -rm -r /pagerank.csv 
ranks.mapPartitions(writeRecords).saveAsTextFile("hdfs://master:9000/pagerank.csv")

rm: `/pagerank.csv': No such file or directory


Good link for PageRank: https://github.com/abbas-taher/pagerank-example-spark2.0-deep-dive

# Part 4: Linear Algebra - Matrix Multiplication

In [32]:
matrixA_txt = sc.textFile('hdfs://master:9000/A.csv')
matrixB_txt = sc.textFile('hdfs://master:9000/B.csv')

In [33]:
matrixA = matrixA_txt.map(lambda line: line.split(","))\
                       .map(lambda lst: (int(lst[0]),int(lst[1]),int(lst[2]), 'A'))

matrixB = matrixB_txt.map(lambda line: line.split(","))\
                       .map(lambda lst: (int(lst[0]),int(lst[1]),int(lst[2]), 'B'))

In [34]:
print(matrixA.first())

(0, 0, 3, 'A')


In [35]:
test = matrixA.filter(lambda tpl: tpl[2] == 0)

In [36]:
vals = matrixA.count()

In [37]:
print(test.count())

909356


The matrix is quite dense. 90% + of the values are not zeros, so we will use the BlockMatrix implementation.

## Option 1: Using Sparks Distributed Matrixes 

In [38]:
from pyspark.mllib.linalg import Matrices
from pyspark.mllib.linalg.distributed import BlockMatrix, CoordinateMatrix, MatrixEntry
from pyspark import sql

sqlContext = sql.SQLContext(sc)

In [39]:
matrixA1 = matrixA.map(lambda tpl: MatrixEntry(tpl[0], tpl[1], tpl[2]))
matrixB1 = matrixB.map(lambda tpl: MatrixEntry(tpl[0], tpl[1], tpl[2]))

In [40]:
print(matrixA1.take(3))

[MatrixEntry(0, 0, 3.0), MatrixEntry(0, 1, 9.0), MatrixEntry(0, 2, 10.0)]


In [41]:
coord_matrixA = CoordinateMatrix(matrixA1)
coord_matrixB = CoordinateMatrix(matrixB1)

In [42]:
BlockMatrixA = coord_matrixA.toBlockMatrix()
BlockMatrixB = coord_matrixB.toBlockMatrix()

In [43]:
result = BlockMatrixA.multiply(BlockMatrixB)

In [44]:
!hadoop fs -rm -r /matrix_mul.csv 
result.toIndexedRowMatrix().rows.sortBy(lambda x: x.index).saveAsTextFile("hdfs://master:9000/matrix_mul.csv")

rm: `/matrix_mul.csv': No such file or directory


## Option 2: Pure MapReduce


In [45]:
matrixA2 = matrixA.map(lambda tpl: (tpl[1],('A', tpl[0], tpl[2])))
matrixB2 = matrixB.map(lambda tpl: (tpl[1],('B', tpl[0], tpl[2])))

In [46]:
joined = matrixA2.join(matrixB2)

In [47]:
print(joined.first())

(712, (('A', 5050, 3), ('B', 0, 6)))


In [48]:
frst_reduce = joined.map(lambda tpl:((tpl[1][0][1]*vals + tpl[1][1][1]), tpl[1][0][2]*tpl[1][1][2]))

In [49]:
print(frst_reduce.first())

(50500000000, 0)


In [50]:
scnd_reduce = frst_reduce.reduceByKey(lambda accum, n: accum + n).map(lambda tpl: ((tpl[0]//vals), (tpl[0]%vals), tpl[1]))

In [51]:
!hadoop fs -rm -r /matrix_mul2.csv 
scnd_reduce.saveAsTextFile("hdfs://master:9000/matrix_mul2.csv")

rm: `/matrix_mul2.csv': No such file or directory
